# MMMI Oversight Script


**TODO**: Configure agents to truly trade.

**TODO**: *Step C.* Quarterly HARK agent updates.

In [1]:
import HARK.ConsumptionSaving.ConsPortfolioModel as cpm
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import sys

sys.path.append('.')

import hark_portfolio_agents as hpa

In [3]:
sys.path.append('../PNL/py')

import util as UTIL
import pnl as pnl

## Initialize the Simulation

### Initialize market information

Some initial values for the simulation.

$RAP_0$ is the starting risky asset price. This is adjust after every trading day.

In [4]:
risky_asset_price = 100

HARK uses numerical values for wealth. This variable converts "HARK money" to dollars.

In [5]:
dollars_per_hark_money_unit = 1000

Expected rate of return and standard deviation of the market price process will be updated over time.

These are the starting values, derived from the S&P 500.

$\bar{r}_0 = 0.000628$

$sr_0 = 0.011988$

In [6]:
sp500_ror = 0.000628
sp500_std = 0.011988

market_rate_of_return = sp500_ror
market_standard_deviation = sp500_std

Calendar variables. Will ultimately set to be realistic; for now set low for testing.

In [7]:
days_per_quarter = 10
quarters_per_simulation = 6

An attention rate: the chance on any day that an agent will pay attention to the market and trade.

Later, this can be a more realistic function of agent circumstances.

In [8]:
attention_rate = 1 / days_per_quarter

We will need to convert daily price statistics to quarterly price statistics. HARK agents use quarterly (?) statistics. [**Question**: Should HARK agents rebalance knowing how many days are left in the quarter? Or based on a mean number of days between rebalancing periods (based on the attention rate)?]

In [9]:
import math

def ror_quarterly(ror, n_q):
    return pow(1 + ror, n_q) - 1

def sig_quarterly(std, n_q):
    """
     The second formula only holds for special cases (see the attached paper by Andrew Lo), 
     but since we are generating the data we meet this special case.
    """
    return math.sqrt(n_q) * std

### Initialize agents to start

This is to set up the agent parameters and quantities used in the simulation.

`CRRA` refers to the `\rho` parameter in the constant relative risk aversion utility function used by the agents. Utility is over units of resources consumed.

`aNrmInitStd` is normalized initial market assets, standard deviation. The agents' starting wealth is drawn from a lognormal distribution with this standard deviation.

`pLvlInitMean` and `pLvlInitStd` are the mean and standard deviation of the initial level of permanent income of the agents.

In [10]:
## List of tuples: 
##  ( parameters, (i.e. coefficient of relative risk aversion CRRA)
##    number of agents represented,
##    ( initial risky percent, $$$ in risky asset, $$$ in riskless asset)
agent_classes = [
    {
        'CRRA': .9,
        'AgentCount' : 10,
        'aNrmInitMean' : 10
    }] * 10 + \
    [{
        'CRRA': .95, 
        'AgentCount' : 10,
        'aNrmInitMean': 10
    }] * 10 + \
    [{
        'CRRA': .99, 
        'AgentCount' : 10,
        'aNrmInitMean' : 10
    }] * 10


### parameters shared by all agents
agent_parameters = {
    'aNrmInitStd' : 0.0,
    'pLvlInitMean' : 1.0, # initial distribution of permanent income
    'pLvlInitStd' : 0.0,
    'Rfree' : 0,
    'RiskyAvg': market_rate_of_return,
    'RiskyStd': market_standard_deviation,    
}

In [11]:
len(agent_classes)

30

Initialize the agents.

In [12]:
agents = hpa.create_agents(agent_classes, agent_parameters)

In [13]:
agents[0].state_now

{'pLvl': array([2.33482594, 2.73191849, 2.33482594, 2.95954583, 3.20232689,
        2.63313315, 2.83445475, 2.63313315, 3.20232689, 2.83445475]),
 'PlvlAgg': 1.0,
 'bNrm': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'mNrm': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'aNrm': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'aLvl': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

## Get agent risky allocations

Use this stock price to determine the initial number of shares of the risky asset held by each
agent:
 - $RAS_{i0} = (HEC_{i0}.RAW)/ RAP_0$
 
 where RAW is Risky Asset Wealth, amount of resources allocated to the risky asset.
 
 RAS is Risky Asset Shares, number of shares the agent has.

In [14]:
def compute_share_demand(agent):
    agent.solve()
    market_resources = agent.state_now['mNrm']
    permanent_income = agent.state_now['pLvl']
    
    print("Market Resources: " + str(market_resources))

    # ShareFunc takes normalized market resources as argument
    risky_share = agent.solution[0].ShareFuncAdj(
        market_resources
    )
    
    # denormalize the risky share. See https://github.com/econ-ark/HARK/issues/986
    risky_asset_wealth = risky_share \
                        * market_resources \
                        * permanent_income \
                        * dollars_per_hark_money_unit 
    
    print("Risky Share: " + str(risky_share))
    print("Risky Asset Wealth: " + str(risky_asset_wealth))
    
    shares = risky_asset_wealth / risky_asset_price
    
    print("Shares: " + str(shares))
    
    return shares

In [15]:
for agent in agents:
    agent.shares = compute_share_demand(agent)

Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13

Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13

## Prepare the main loop

Functions to automate aspects of the main loop.
(Will move this to module code once finalized).

### Data stores

For prices, rates or return, and standard deviations over time.


In [16]:

prices = []
ror_list = []
std_list = []

expected_ror_list = []
expected_std_list = []

buy_sell_list = []

### Activating agents

"At the beginning of each trading day $t + 1$, select a random group of HARK agents with replacement."

**TODO**: HARK AgentType classes represent multiple agents (as many as `AgentCount`). It is difficult to have the agents within the same AgentType act 'separately', though these agents will have _ex ante_ and _ex post_ heterogeneity of some variables. For the purpose of MMMI, we may need to scaffold the agent model to make it easier to have multiple independently acting agents.

In [17]:
import random

def activate(agents, attention_rate):
    for agent in agents:
        agent.attentive = random.random() < attention_rate

In [18]:
activate(agents, attention_rate)

"**First**, determine the expected return and standard deviation of the risky asset to be used for decision period $t + 1$."

A number of constants used in the calculation of risky expecations.

In [19]:
import math

p1 = 0.1
delta_t1 = 30

a = - math.log(p1) / delta_t1

p2 = 0.1
delta_t2 = 60

b = math.log(p2) / delta_t2

In [20]:
def calculate_risky_expectations():
    # note use of data store lists for time tracking here -- not ideal
    D_t = sum([math.exp(a * l) for l in ror_list])
    S_t = math.exp(b * len(prices))
    
    w_0 = S_t
    w_t = [(1 - S_t) * math.exp(a * (t+1)) / D_t for t in range(len(ror_list))]
    
    expected_ror = w_0 * sp500_ror + sum(
        [w_ror[0] * w_ror[1]
         for w_ror
         in zip(w_t, ror_list)])
    expected_ror_list.append(expected_ror)
    
    expected_std = math.sqrt(
        w_0 * pow(sp500_std, 2) \
        +  sum([w_ror_er[0] * pow(w_ror_er[1] - w_ror_er[2], 2)
                for w_ror_er
                in zip(w_t, ror_list, expected_ror_list)]))
    expected_std_list.append(expected_std)

    # TODO: This should be an adaptive function of market prices
    # as described in John's Experiment design.
    # This is a stopgap for now.
    market_risky_params = {
        'RiskyAvg': ror_quarterly(expected_ror, days_per_quarter),
        'RiskyStd': sig_quarterly(expected_std, days_per_quarter)
    }
    
    return market_risky_params


"**Second**, using [these expectations] calculate the total number of shares of the risky asset that the selected agents want to buy and sell."

In [21]:
def compute_total_buy_sell(agents):
    """
    TODO: Should this be done without making so many changes to the underlying
    agent? Could be done using copies.
    """
    risky_expectations = calculate_risky_expectations()
    
    buy_shares_total = 0
    sell_shares_total = 0
    
    for agent in agents:
        if agent.attentive:
            # Note: this mutates the underlying agent
            agent.assign_parameters(**risky_expectations)
            
            d_shares = compute_share_demand(agent)
            
            delta_shares = d_shares - agent.shares
            
            # NOTE: This mutates the agent
            agent.shares = d_shares
            
            buy_shares_total += delta_shares[delta_shares > 0].sum()
            sell_shares_total += -delta_shares[delta_shares < 0].sum()
            
    return (round(buy_shares_total), round(sell_shares_total))

In [22]:
compute_total_buy_sell(agents)

Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]


(0, 0)

"**Third**, using [total buy and sell shares] as input to the Buy Broker and Sell Broker, run a day of NetLogo trading and determine the end of day price of the risky asset."

In [23]:
import os

def run_market(config, buy_sell, seed = None):    
    pnl.run_NLsims(
        config,
        broker_buy_limit = buy_sell[0],
        broker_sell_limit = buy_sell[1],
        SEED = seed,
        use_cache = True
    )

"**Fourth**, use the final price to calculate the amount of wealth each (all) HARK agent has allocated to the risky asset."

In [24]:
def get_transactions(config, seed = 0, buy_sell = (0,0)):
    logfile = pnl.transaction_file_name(config, seed, buy_sell[0], buy_sell[1])
    
    # use run_market() first to create logs
    transactions = pd.read_csv(
        logfile,
        delimiter='\t'
    )
    return transactions

def get_last_simulation_price(config, seed = 0, buy_sell = (0,0)):
    """
    Dividing the output of this by 4
    """
    
    transactions = get_transactions(config, seed=seed, buy_sell = buy_sell)
    prices = transactions['TrdPrice']
    
    return prices[prices.index.values[-1]]

In [25]:
netlogo_ror = -0.00052125
netlogo_std =  0.0068
simulation_price_scale = 0.25

def daily_rate_of_return(config = None, seed = 0, buy_sell = (0, 0)):
    
    last_sim_price = get_last_simulation_price(config, seed=seed, buy_sell = buy_sell)
    
    ror = (last_sim_price * simulation_price_scale - 100) / 100
    
    # adjust to calibrated NetLogo to S&P500
    ror = sp500_std * (ror - netlogo_ror) / netlogo_std + sp500_ror
    
    return ror

In [26]:
def update_agent_wealth(old_share_price, ror):

    new_share_price = old_share_price * (1 + ror)

    for agent in agents:
        market_resources = agent.state_now['mNrm'] * agent.state_now['pLvl']
    
        old_raw = agent.shares * old_share_price
        new_raw = agent.shares * new_share_price
    
        delta_mNrm = (new_raw - old_raw) / \
            (dollars_per_hark_money_unit * agent.state_now['pLvl'])
    
        agent.state_now['mNrm'] = agent.state_now['mNrm'] + delta_mNrm

"Repeat each trading day until the end of the quarter. Then update the economic conditions for each HARK agent (e.g., permanent income level), and repeat [daily loop]."

## Run the main loop



In [27]:
import itertools

seeds = itertools.cycle([0,1,2,3,4])

config = UTIL.read_config(
    config_file = "../PNL/macroliquidity.ini",
    config_local_file = "../PNL/macroliquidity_local.ini"
)

# Main loop
for quarter in range(quarters_per_simulation):
    for day in range(days_per_quarter):
        print(f"Q-{quarter}:D-{day}")
        
        # Set to a number for a fixed seed, or None to rotate
        seed = next(seeds)

        prices.append(risky_asset_price)
        
        activate(agents, attention_rate)
        buy_sell = compute_total_buy_sell(agents)
        print("Buy/Sell Limit: " + str(buy_sell))
        buy_sell_list.append(buy_sell)
        
        run_market(config, buy_sell, seed = seed)
        
        ror =  daily_rate_of_return(config = config, seed = seed, buy_sell = buy_sell)
        print("ror: " + str(ror))
        ror_list.append(ror)
        
        update_agent_wealth(risky_asset_price, ror)
        
        risky_asset_price = risky_asset_price * (1 + ror)

config_file: ../PNL/macroliquidity.ini
config_local_file: ../PNL/macroliquidity_local.ini
Q-0:D-0
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.82593888 2731.91848655 2334.82593888 2959.54583051 3202.32688908
 2633.13315336 2834.45474991 2633.13315336 3202.32688908 2834.45474991]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2334.825

Market Resources: [1.02879039 1.02879039 1.02879039 1.02879039 1.02879039 1.02879039
 1.02879039 1.02879039 1.02879039 1.02879039]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2402.04648169 2810.57147756 2402.04648169 3044.75230089 3294.52312013
 2708.94207638 2916.05979964 2708.94207638 3294.52312013 2916.05979964]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.02879039 1.02879039 1.02879039 1.02879039 1.02879039 1.02879039
 1.02879039 1.02879039 1.02879039 1.02879039]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2402.04648169 2810.57147756 2402.04648169 3044.75230089 3294.52312013
 2708.94207638 2916.05979964 2708.94207638 3294.52312013 2916.05979964]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.02879039 1.02879039 1.02879039 1.028790

Market Resources: [1.05051981 1.05051981 1.05051981 1.05051981 1.05051981 1.05051981
 1.05051981 1.05051981 1.05051981 1.05051981]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2452.78089076 2869.93447664 2452.78089076 3109.0615097  3364.10782008
 2766.15852765 2977.65085206 2766.15852765 3364.10782008 2977.65085206]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:2,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Q-1:D-3
Market Resources: [1.0567749 1.0567749 1.0567749 1.0567749 1.0567749 1.0567749 1.0567749
 1.0567749 1.0567749 1.0567749]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2467.38544975 2887.0228874  2467.38544975 3127.57375122 3384.13868027
 2782.62902672 2995.38063693 2782.62902672 3384.13868027 2995.38063693]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.34

Market Resources: [1.07105133 1.07105133 1.07105133 1.07105133 1.07105133 1.07105133
 1.07105133 1.07105133 1.07105133 1.07105133]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2500.71843379 2926.02493624 2500.71843379 3169.82550638 3429.85648274
 2820.22077346 3035.84653777 2820.22077346 3429.85648274 3035.84653777]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.07105133 1.07105133 1.07105133 1.07105133 1.07105133 1.07105133
 1.07105133 1.07105133 1.07105133 1.07105133]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2500.71843379 2926.02493624 2500.71843379 3169.82550638 3429.85648274
 2820.22077346 3035.84653777 2820.22077346 3429.85648274 3035.84653777]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:1,BL:0,SL:0 already exis

Market Resources: [1.09367336 1.09367336 1.09367336 1.09367336 1.09367336 1.09367336
 1.09367336 1.09367336 1.09367336 1.09367336]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2553.53692543 2987.82646557 2553.53692543 3236.77642726 3502.29960289
 2879.78757847 3099.96764505 2879.78757847 3502.29960289 3099.96764505]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.09367336 1.09367336 1.09367336 1.09367336 1.09367336 1.09367336
 1.09367336 1.09367336 1.09367336 1.09367336]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2553.53692543 2987.82646557 2553.53692543 3236.77642726 3502.29960289
 2879.78757847 3099.96764505 2879.78757847 3502.29960289 3099.96764505]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.09367336 1.09367336 1.09367336 1.093673

...

Market Resources: [1.14892552 1.14892552 1.14892552 1.14892552 1.14892552 1.14892552
 1.14892552 1.14892552 1.14892552 1.14892552]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2682.54111378 3138.77087693 2682.54111378 3400.29774223 3679.235097
 3025.2738863  3256.57740697 3025.2738863  3679.235097   3256.57740697]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.14892552 1.14892552 1.14892552 1.14892552 1.14892552 1.14892552
 1.14892552 1.14892552 1.14892552 1.14892552]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2682.54111378 3138.77087693 2682.54111378 3400.29774223 3679.235097
 3025.2738863  3256.57740697 3025.2738863  3679.235097   3256.57740697]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.14892552 1.14892552 1.14892552 1.14892552 1

Market Resources: [1.17500721 1.17500721 1.17500721 1.17500721 1.17500721 1.17500721
 1.17500721 1.17500721 1.17500721 1.17500721]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2743.43730071 3210.0239053  2743.43730071 3477.48767452 3762.75716758
 3093.95042704 3330.50475352 3093.95042704 3762.75716758 3330.50475352]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:4,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Q-4:D-0
Market Resources: [1.18200353 1.18200353 1.18200353 1.18200353 1.18200353 1.18200353
 1.18200353 1.18200353 1.18200353 1.18200353]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2759.7725111  3229.13730579 2759.7725111  3498.1936308  3785.16170001
 3112.37269285 3350.33553144 3112.37269285 3785.16170001 3350.33553144]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133

Market Resources: [1.21603387 1.21603387 1.21603387 1.21603387 1.21603387 1.21603387
 1.21603387 1.21603387 1.21603387 1.21603387]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2839.22743026 3322.10541912 2839.22743026 3598.9079799  3894.13797094
 3201.97910776 3446.79298862 3201.97910776 3894.13797094 3446.79298862]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:2,BL:0,SL:0 already exists.
Will use cache.
ror: 0.005954286029411765
Q-4:D-8
Market Resources: [1.22327449 1.22327449 1.22327449 1.22327449 1.22327449 1.22327449
 1.22327449 1.22327449 1.22327449 1.22327449]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2856.13300248 3341.88618501 2856.13300248 3620.33690741 3917.32478226
 3221.04460723 3467.31617996 3221.04460723 3917.32478226 3467.31617996]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133

Market Resources: [1.24171812 1.24171812 1.24171812 1.24171812 1.24171812 1.24171812
 1.24171812 1.24171812 1.24171812 1.24171812]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2899.19568109 3392.27269383 2899.19568109 3674.92169199 3976.3873322
 3269.60915537 3519.59383024 3269.60915537 3976.3873322  3519.59383024]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.24171812 1.24171812 1.24171812 1.24171812 1.24171812 1.24171812
 1.24171812 1.24171812 1.24171812 1.24171812]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2899.19568109 3392.27269383 2899.19568109 3674.92169199 3976.3873322
 3269.60915537 3519.59383024 3269.60915537 3976.3873322  3519.59383024]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:2,BL:0,SL:0 already exists

Market Resources: [1.26598646 1.26598646 1.26598646 1.26598646 1.26598646 1.26598646
 1.26598646 1.26598646 1.26598646 1.26598646]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2955.85802636 3458.5718153  2955.85802636 3746.74495081 4054.10248382
 3333.51092098 3588.38133642 3333.51092098 4054.10248382 3588.38133642]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Market Resources: [1.26598646 1.26598646 1.26598646 1.26598646 1.26598646 1.26598646
 1.26598646 1.26598646 1.26598646 1.26598646]
Risky Share: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Risky Asset Wealth: [2955.85802636 3458.5718153  2955.85802636 3746.74495081 4054.10248382
 3333.51092098 3588.38133642 3333.51092098 4054.10248382 3588.38133642]
Shares: [23.34825939 27.31918487 23.34825939 29.59545831 32.02326889 26.33133153
 28.3445475  26.33133153 32.02326889 28.3445475 ]
Buy/Sell Limit: (0, 0)
Output for S:1,BL:0,SL:0 already exis

In [28]:
len(ror_list)

60

In [30]:
pd.DataFrame.from_dict({
    'prices' : [100] + prices,
    'buy' : [None] + [bs[0] for bs in buy_sell_list],
    'sell' : [None]  + [bs[1] for bs in buy_sell_list],
    'ror_list' : [None] + ror_list,
    'expected_ror' : expected_ror_list,
    'expected_ror_q' : [ror_quarterly(er, days_per_quarter) for er in expected_ror_list],
    'expected_std' : expected_std_list,
    'expected_std_q' : [sig_quarterly(es, days_per_quarter) for es in expected_std_list],
})




,prices,buy,sell,ror_list,expected_ror,expected_ror_q,expected_std,expected_std_q
0,100.000000,NaN,NaN,NaN,0.000628,0.006298,0.011988,0.037909
1,100.000000,0.0,0.0,0.005954,0.000604,0.006060,0.011760,0.037189
2,100.595429,0.0,0.0,0.001547,0.001056,0.010614,0.011634,0.036791
3,100.751043,0.0,0.0,0.005954,0.001007,0.010118,0.011393,0.036028
4,101.350944,0.0,0.0,0.001547,0.001284,0.012919,0.011234,0.035524
...,...,...,...,...,...,...,...,...
56,125.849303,0.0,0.0,0.005954,0.061298,0.812895,0.116928,0.369758
57,126.598646,0.0,0.0,0.001547,0.067442,0.920633,0.127897,0.404446
58,126.794486,0.0,0.0,0.005954,0.068495,0.939662,0.143469,0.453690
59,127.549456,0.0,0.0,0.001547,0.075507,1.070770,0.159150,0.503276
